In [4]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/yy2895/Desktop/update_stresult19-14-19.csv',header=None)
d = df.values
ntotal=len(df)

# we do not renormalize it 
#d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(7)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,8):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 14
n_encoder_h_2 = 9



#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 14
n_decoder_h_2 = 19




# Parameters
learning_rate = 0.005

#batch_size = 7
display_step = 1

total_batch=7
training_epochs = 5000

In [8]:
#we store the variables here
log_files_path = 'C:/Users/yy2895/Desktop/st14-10-14/tmp/model.ckpt'
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        
        with tf.variable_scope("code"):
            output = layer(x, [14, n_code], [n_code], phase_train)


    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
   
        with tf.variable_scope("output"):
            output = layer(x, [ n_code, 14], [14], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '10'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 14]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive10=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/yy2895/Desktop/st14-10-14/tmp/model.ckpt')
                        togive3=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,14)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive3.append(output_any_image)
                            
                        togive10=togive3
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)
            

we begin
Epoch: 0001 cost = 1.477621658
Validation Loss: 1.7107922
Epoch: 0002 cost = 1.458867993
Validation Loss: 1.6648624
Epoch: 0003 cost = 1.441209946
Validation Loss: 1.5608352
Epoch: 0004 cost = 1.423821875
Validation Loss: 1.516264
Epoch: 0005 cost = 1.406043410
Validation Loss: 1.4897624
Epoch: 0006 cost = 1.388577700
Validation Loss: 1.4662479
Epoch: 0007 cost = 1.372014727
Validation Loss: 1.4344769
Epoch: 0008 cost = 1.356213604
Validation Loss: 1.4100777
Epoch: 0009 cost = 1.341070703
Validation Loss: 1.3938932
Epoch: 0010 cost = 1.326515998
Validation Loss: 1.3718297
Epoch: 0011 cost = 1.312378815
Validation Loss: 1.3530461
Epoch: 0012 cost = 1.298532673
Validation Loss: 1.3393945
Epoch: 0013 cost = 1.284949354
Validation Loss: 1.3247288
Epoch: 0014 cost = 1.271744268
Validation Loss: 1.3120531
Epoch: 0015 cost = 1.259104235
Validation Loss: 1.2963032
Epoch: 0016 cost = 1.247097441
Validation Loss: 1.2779834
Epoch: 0017 cost = 1.235615662
Validation Loss: 1.2560174
Epoch:

Epoch: 0150 cost = 0.478704308
Validation Loss: 0.5312332
Epoch: 0151 cost = 0.475225670
Validation Loss: 0.48282394
Epoch: 0152 cost = 0.471778904
Validation Loss: 0.46818602
Epoch: 0153 cost = 0.468362940
Validation Loss: 0.49305475
Epoch: 0154 cost = 0.464977754
Validation Loss: 0.5406811
Epoch: 0155 cost = 0.461622374
Validation Loss: 0.47731873
Epoch: 0156 cost = 0.458295728
Validation Loss: 0.4245923
Epoch: 0157 cost = 0.454997561
Validation Loss: 0.56960374
Epoch: 0158 cost = 0.451727424
Validation Loss: 0.44081053
Epoch: 0159 cost = 0.448483965
Validation Loss: 0.43207896
Epoch: 0160 cost = 0.445267809
Validation Loss: 0.36091083
Epoch: 0161 cost = 0.442077777
Validation Loss: 0.4587518
Epoch: 0162 cost = 0.438913912
Validation Loss: 0.37674206
Epoch: 0163 cost = 0.435776357
Validation Loss: 0.4033853
Epoch: 0164 cost = 0.432664509
Validation Loss: 0.46771216
Epoch: 0165 cost = 0.429578270
Validation Loss: 0.5331566
Epoch: 0166 cost = 0.426517593
Validation Loss: 0.53056705
Epo

Epoch: 0297 cost = 0.181336803
Validation Loss: 0.16674986
Epoch: 0298 cost = 0.180330519
Validation Loss: 0.12897049
Epoch: 0299 cost = 0.179334447
Validation Loss: 0.14905226
Epoch: 0300 cost = 0.178348490
Validation Loss: 0.13620211
Epoch: 0301 cost = 0.177372577
Validation Loss: 0.13036627
Epoch: 0302 cost = 0.176406024
Validation Loss: 0.13494484
Epoch: 0303 cost = 0.175448907
Validation Loss: 0.13790295
Epoch: 0304 cost = 0.174501123
Validation Loss: 0.13899441
Epoch: 0305 cost = 0.173562518
Validation Loss: 0.16587202
Epoch: 0306 cost = 0.172632662
Validation Loss: 0.1725166
Epoch: 0307 cost = 0.171711519
Validation Loss: 0.18208444
Epoch: 0308 cost = 0.170799145
Validation Loss: 0.17970791
Epoch: 0309 cost = 0.169894929
Validation Loss: 0.14847215
Epoch: 0310 cost = 0.168999295
Validation Loss: 0.14017133
Epoch: 0311 cost = 0.168111776
Validation Loss: 0.13636804
Epoch: 0312 cost = 0.167232407
Validation Loss: 0.14295736
Epoch: 0313 cost = 0.166360996
Validation Loss: 0.1375285

Epoch: 0438 cost = 0.098809350
Validation Loss: 0.0952519
Epoch: 0439 cost = 0.099538411
Validation Loss: 0.09328842
Epoch: 0440 cost = 0.098186277
Validation Loss: 0.087892294
Epoch: 0441 cost = 0.098932534
Validation Loss: 0.08885803
Epoch: 0442 cost = 0.097569643
Validation Loss: 0.11665178
Epoch: 0443 cost = 0.098334184
Validation Loss: 0.08808568
Epoch: 0444 cost = 0.096959136
Validation Loss: 0.091255605
Epoch: 0445 cost = 0.097742854
Validation Loss: 0.07910977
Epoch: 0446 cost = 0.096354556
Validation Loss: 0.08678697
Epoch: 0447 cost = 0.097158613
Validation Loss: 0.07547068
Epoch: 0448 cost = 0.095755569
Validation Loss: 0.10675319
Epoch: 0449 cost = 0.096581116
Validation Loss: 0.08497336
Epoch: 0450 cost = 0.095162043
Validation Loss: 0.098439485
Epoch: 0451 cost = 0.096010150
Validation Loss: 0.08767751
Epoch: 0452 cost = 0.094574096
Validation Loss: 0.0985533
Epoch: 0453 cost = 0.095445207
Validation Loss: 0.08140134
Epoch: 0454 cost = 0.093991186
Validation Loss: 0.10078

Epoch: 0579 cost = 0.064462357
Validation Loss: 0.064053066
Epoch: 0580 cost = 0.063089115
Validation Loss: 0.0625979
Epoch: 0581 cost = 0.064010764
Validation Loss: 0.060268722
Epoch: 0582 cost = 0.062671959
Validation Loss: 0.080543764
Epoch: 0583 cost = 0.063558896
Validation Loss: 0.05622928
Epoch: 0584 cost = 0.062255388
Validation Loss: 0.08727768
Epoch: 0585 cost = 0.063105603
Validation Loss: 0.05893024
Epoch: 0586 cost = 0.061837254
Validation Loss: 0.055485576
Epoch: 0587 cost = 0.062649118
Validation Loss: 0.06247819
Epoch: 0588 cost = 0.061416688
Validation Loss: 0.06353625
Epoch: 0589 cost = 0.062188564
Validation Loss: 0.070001625
Epoch: 0590 cost = 0.060992319
Validation Loss: 0.07001673
Epoch: 0591 cost = 0.061722081
Validation Loss: 0.057865698
Epoch: 0592 cost = 0.060563274
Validation Loss: 0.059112377
Epoch: 0593 cost = 0.061247984
Validation Loss: 0.052014787
Epoch: 0594 cost = 0.060126983
Validation Loss: 0.075208366
Epoch: 0595 cost = 0.060765511
Validation Loss: 

...

Validation Loss: 0.025562499
Epoch: 4339 cost = 0.012765934
Validation Loss: 0.022352984
Epoch: 4340 cost = 0.012765854
Validation Loss: 0.016394503
Epoch: 4341 cost = 0.012765426
Validation Loss: 0.023712933
Epoch: 4342 cost = 0.012764846
Validation Loss: 0.025857318
Epoch: 4343 cost = 0.012764199
Validation Loss: 0.030206356
Epoch: 4344 cost = 0.012764614
Validation Loss: 0.034535903
Epoch: 4345 cost = 0.012764227
Validation Loss: 0.026760295
Epoch: 4346 cost = 0.012763290
Validation Loss: 0.023544239
Epoch: 4347 cost = 0.012763431
Validation Loss: 0.030191185
Epoch: 4348 cost = 0.012763697
Validation Loss: 0.024567604
Epoch: 4349 cost = 0.012762222
Validation Loss: 0.02042953
Epoch: 4350 cost = 0.012762207
Validation Loss: 0.023639392
Epoch: 4351 cost = 0.012762053
Validation Loss: 0.01873486
Epoch: 4352 cost = 0.012762027
Validation Loss: 0.023469353
Epoch: 4353 cost = 0.012761488
Validation Loss: 0.027068613
Epoch: 4354 cost = 0.012760586
Validation Loss: 0.020280754
Epoch: 4355 c

Validation Loss: 0.015836367
Epoch: 4477 cost = 0.012720719
Validation Loss: 0.02491212
Epoch: 4478 cost = 0.012720729
Validation Loss: 0.029922502
Epoch: 4479 cost = 0.012720162
Validation Loss: 0.031019831
Epoch: 4480 cost = 0.012719658
Validation Loss: 0.026403919
Epoch: 4481 cost = 0.012719422
Validation Loss: 0.021727327
Epoch: 4482 cost = 0.012719132
Validation Loss: 0.017109016
Epoch: 4483 cost = 0.012718555
Validation Loss: 0.020447597
Epoch: 4484 cost = 0.012718153
Validation Loss: 0.015410351
Epoch: 4485 cost = 0.012718419
Validation Loss: 0.018354286
Epoch: 4486 cost = 0.012717820
Validation Loss: 0.01924567
Epoch: 4487 cost = 0.012717121
Validation Loss: 0.026098976
Epoch: 4488 cost = 0.012716578
Validation Loss: 0.026611902
Epoch: 4489 cost = 0.012716882
Validation Loss: 0.02946282
Epoch: 4490 cost = 0.012716047
Validation Loss: 0.020211283
Epoch: 4491 cost = 0.012716098
Validation Loss: 0.021142986
Epoch: 4492 cost = 0.012716136
Validation Loss: 0.031274404
Epoch: 4493 co

Epoch: 4629 cost = 0.012668537
Validation Loss: 0.02461512
Epoch: 4630 cost = 0.012668478
Validation Loss: 0.0219525
Epoch: 4631 cost = 0.012667321
Validation Loss: 0.022533517
Epoch: 4632 cost = 0.012666825
Validation Loss: 0.02357976
Epoch: 4633 cost = 0.012666770
Validation Loss: 0.027446799
Epoch: 4634 cost = 0.012666282
Validation Loss: 0.022721753
Epoch: 4635 cost = 0.012666488
Validation Loss: 0.03150489
Epoch: 4636 cost = 0.012665859
Validation Loss: 0.046802007
Epoch: 4637 cost = 0.012666041
Validation Loss: 0.027516063
Epoch: 4638 cost = 0.012665248
Validation Loss: 0.030482521
Epoch: 4639 cost = 0.012664935
Validation Loss: 0.026893238
Epoch: 4640 cost = 0.012664567
Validation Loss: 0.032622263
Epoch: 4641 cost = 0.012664439
Validation Loss: 0.026720246
Epoch: 4642 cost = 0.012663182
Validation Loss: 0.025349319
Epoch: 4643 cost = 0.012664276
Validation Loss: 0.02566419
Epoch: 4644 cost = 0.012663145
Validation Loss: 0.027289446
Epoch: 4645 cost = 0.012663012
Validation Loss

Validation Loss: 0.024675554
Epoch: 4776 cost = 0.012616748
Validation Loss: 0.01951176
Epoch: 4777 cost = 0.012616694
Validation Loss: 0.017296875
Epoch: 4778 cost = 0.012616218
Validation Loss: 0.032971825
Epoch: 4779 cost = 0.012615832
Validation Loss: 0.041701626
Epoch: 4780 cost = 0.012615461
Validation Loss: 0.02741125
Epoch: 4781 cost = 0.012614933
Validation Loss: 0.019141994
Epoch: 4782 cost = 0.012614615
Validation Loss: 0.021843031
Epoch: 4783 cost = 0.012614417
Validation Loss: 0.023860384
Epoch: 4784 cost = 0.012613660
Validation Loss: 0.023776403
Epoch: 4785 cost = 0.012613653
Validation Loss: 0.029543499
Epoch: 4786 cost = 0.012613652
Validation Loss: 0.024430681
Epoch: 4787 cost = 0.012613251
Validation Loss: 0.03211078
Epoch: 4788 cost = 0.012612774
Validation Loss: 0.032458093
Epoch: 4789 cost = 0.012612835
Validation Loss: 0.031525612
Epoch: 4790 cost = 0.012611803
Validation Loss: 0.024512097
Epoch: 4791 cost = 0.012611274
Validation Loss: 0.019670723
Epoch: 4792 co

Validation Loss: 0.045671377
Epoch: 4917 cost = 0.012569194
Validation Loss: 0.045698278
Epoch: 4918 cost = 0.012568767
Validation Loss: 0.03610134
Epoch: 4919 cost = 0.012568378
Validation Loss: 0.031908605
Epoch: 4920 cost = 0.012568311
Validation Loss: 0.028622642
Epoch: 4921 cost = 0.012567682
Validation Loss: 0.033938188
Epoch: 4922 cost = 0.012567111
Validation Loss: 0.024154074
Epoch: 4923 cost = 0.012566952
Validation Loss: 0.03139339
Epoch: 4924 cost = 0.012566856
Validation Loss: 0.040120814
Epoch: 4925 cost = 0.012566593
Validation Loss: 0.039700355
Epoch: 4926 cost = 0.012565651
Validation Loss: 0.030699886
Epoch: 4927 cost = 0.012565563
Validation Loss: 0.019118099
Epoch: 4928 cost = 0.012565629
Validation Loss: 0.02678111
Epoch: 4929 cost = 0.012565058
Validation Loss: 0.032560274
Epoch: 4930 cost = 0.012564464
Validation Loss: 0.035471704
Epoch: 4931 cost = 0.012564338
Validation Loss: 0.02270541
Epoch: 4932 cost = 0.012564108
Validation Loss: 0.016725585
Epoch: 4933 cos

In [9]:
currentmin

0.010266256

In [10]:
import csv
with open('C:/Users/yy2895/Desktop/update_stresult14-10-14.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive3)):
        wr.writerow(list(togive3[i][0]))

In [11]:
togive3

[array([[0.18888506, 0.19898792, 0.329743  , 0.72397375, 0.139016  ,
         0.77916986, 0.74029064, 0.347895  , 0.37947345, 0.86553514]],
       dtype=float32),
 array([[0.18292631, 0.17480557, 0.34514594, 0.7184445 , 0.1296512 ,
         0.7831607 , 0.741733  , 0.35088143, 0.37724173, 0.86885804]],
       dtype=float32),
 array([[0.17416574, 0.17346913, 0.30567208, 0.7179786 , 0.12849899,
         0.78538924, 0.7478609 , 0.33251756, 0.38244033, 0.8679365 ]],
       dtype=float32),
 array([[0.1734374 , 0.19809288, 0.3035711 , 0.70983046, 0.12791628,
         0.7880835 , 0.7417218 , 0.31291392, 0.3987469 , 0.86572725]],
       dtype=float32),
 array([[0.167542  , 0.23115537, 0.26456735, 0.70091295, 0.13516732,
         0.78852284, 0.7425108 , 0.2774203 , 0.41774914, 0.8648367 ]],
       dtype=float32),
 array([[0.17022698, 0.21831554, 0.2936367 , 0.69640285, 0.1271238 ,
         0.78976476, 0.74427676, 0.30264708, 0.4115637 , 0.86856896]],
       dtype=float32),
 array([[0.16287716, 0